In [ ]:
import os
import numpy as np
import cv2

from matplotlib import pyplot as plt
import matplotlib 

from time import time
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

### Cargado de datos

Organizamos las imágenes en carpetas para poder procesarlas correctamente

In [15]:
import pandas as pd
from shutil import copyfile

csv_path = "C:/Users/lllrm/Downloads/dataset_VC/fairface_label_train.csv"
output_dir = "C:/Users/lllrm/Downloads/dataset_VC/ages"
image_path = "C:/Users/lllrm/Downloads/dataset_VC"

df = pd.read_csv(csv_path)

for _, row in df.iterrows():
    age_group = row['age']
    src = os.path.join(image_path, row['file'])  # original image path
    dst = os.path.join(output_dir, age_group, os.path.basename(row['file']))  # ignore train/test subfolder

    # Make sure the destination folder exists
    os.makedirs(os.path.dirname(dst), exist_ok=True)

    # Copy the file if it exists
    if os.path.exists(src):
        copyfile(src, dst)

In [ ]:
folder = "C:/Users/lllrm/Downloads/dataset_VC/train"

nclasses = 0
nperclass = []
classlabels = []
X = []
Y = []

dpi = matplotlib.rcParams['figure.dpi']

for class_name in os.listdir(folder):
    nclasses += 1
    nsamples = 0

    class_folder = os.path.join(folder, class_name)
    for file_name in os.listdir(class_folder):
        if file_name.endswith('.jpg'):
            image = cv2.imread (os.path.join(class_folder, file_name),)
            image = cv2.resize(image, (640, 640))
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            new_height, new_width = gray.shape

            X.append(gray.reshape(new_height*new_width))
            Y.append(nclasses-1)
  
            nsamples += 1
    nperclass.append(nsamples)
    classlabels.append(class_name)
X = np.array(X,dtype='float32')
Y = np.array(Y,dtype='float64')


n_samples , n_features = X.shape

class_names = np.array(classlabels)
n_classes = class_names.shape[0]

print("Dataset info:")
print("# features: %d" % n_features)
print("# classes: %d" % n_classes)
print("classes %s" % classlabels)
print("samples per class %s" % str(nperclass)[1:-1] )

Processing class '1.jpg' 


NotADirectoryError: [WinError 267] The directory name is invalid: 'C:/Users/lllrm/Downloads/dataset_VC/train\\1.jpg'

## Entrenamiento